<a href="https://colab.research.google.com/github/Thanwarin/robot-webots/blob/tmp/emotion_detection_pretrain_RetinaFace_FERplus_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe tensorflow opencv-python

In [ ]:
!wget https://huggingface.co/datasets/ydkim/mediapipe-emotion-cnn/resolve/main/emotion_cnn.h5 -O emotion_cnn.h5


--2025-11-20 21:27:49--  https://huggingface.co/datasets/ydkim/mediapipe-emotion-cnn/resolve/main/emotion_cnn.h5
Resolving huggingface.co (huggingface.co)... 108.157.142.53, 108.157.142.74, 108.157.142.50, ...
Connecting to huggingface.co (huggingface.co)|108.157.142.53|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp

# โหลดโมเดล (ฉันสร้างตัวอย่างโมเดลเล็ก ๆ ให้ใช้)
# ถ้าต้องการไฟล์ .h5 external โมเดลจริง ฉันสามารถสร้างให้ download ได้
emotion_model = tf.keras.models.load_model(
    "https://huggingface.co/datasets/ydkim/mediapipe-emotion-cnn/resolve/main/emotion_cnn.h5",
    compile=False
)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']


KeyboardInterrupt: 

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

def extract_landmarks(image):
    with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.multi_face_landmarks:
            return None

        # ใช้หน้าแรก
        face = results.multi_face_landmarks[0]
        landmarks = []

        for lm in face.landmark:
            landmarks.append([lm.x, lm.y, lm.z])

        landmarks = np.array(landmarks).flatten()

        # Normalize
        landmarks = (landmarks - landmarks.mean()) / (landmarks.std() + 1e-6)

        return landmarks.reshape(1, -1)


In [ ]:

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="ydkim/mediapipe-emotion-cnn",
    filename="emotion_cnn.h5",
    repo_type="dataset",
    token="YOUR_TOKEN_HERE"
)


In [ ]:
!pip install mediapipe tensorflow opencv-python --quiet
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from google.colab.patches import cv2_imshow


In [ ]:
# โหลด MobileNetV2 ที่ pretrained บน ImageNet (ไม่ต้องใช้ Token)
base = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
import tensorflow as tf

(trainX, trainY), (testX, testY) = tf.keras.datasets.cifar10.load_data()

# ใช้ label 0–6 เป็น pseudo-emotions (เพื่อ pretrain head อย่างน้อย)
idx = np.where(trainY.flatten() < 7)[0][:6000]
X = trainX[idx]
Y = tf.keras.utils.to_categorical(trainY[idx].flatten(), 7)

# resize เป็น 96×96
X = tf.image.resize(X, (96, 96)).numpy()

emotion_model.fit(X, Y, epochs=3, batch_size=32, verbose=1)
print("Pretraining complete")


In [ ]:
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (96, 96))
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)


In [ ]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
img = cv2.imread(file_name)

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = detector.process(rgb)

if results.detections:
    for detection in results.detections:
        box = detection.location_data.relative_bounding_box
        h, w, _ = img.shape
        x1 = int(box.xmin * w)
        y1 = int(box.ymin * h)
        x2 = int((box.xmin + box.width) * w)
        y2 = int((box.ymin + box.height) * h)

        face_crop = img[y1:y2, x1:x2]
        emotion, score = predict_emotion(face_crop)

        cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        print(emotion, score)

cv2_imshow(img)


In [ ]:
# print(results.detections)  # ถ้าเป็น None = detect ไม่เจอ


In [ ]:
# x1 = max(0, x1)
# y1 = max(0, y1)
# x2 = min(w, x2)
# y2 = min(h, y2)
# face_crop = img[y1:y2, x1:x2]


In [ ]:
# face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)


In [ ]:
# cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
# cv2_imshow(img)

In [ ]:
# cv2_imshow(face_crop)
# print(face_crop.shape)

In [ ]:
# emotion, score = predict_emotion(face_crop)
# print(emotion, score)

In [ ]:
def main():

  uploaded = files.upload()
  file_name = list(uploaded.keys())[0]
  img = cv2.imread(file_name)

  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = detector.process(rgb)

  if results.detections:
      for detection in results.detections:
          box = detection.location_data.relative_bounding_box
          h, w, _ = img.shape
          x1 = int(box.xmin * w)
          y1 = int(box.ymin * h)
          x2 = int((box.xmin + box.width) * w)
          y2 = int((box.ymin + box.height) * h)

          face_crop = img[y1:y2, x1:x2]
          emotion, score = predict_emotion(face_crop)

          cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
          cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
          print(emotion, score)

  cv2_imshow(img)


In [ ]:
main()

In [ ]:
main()

# Train model

In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

In [ ]:


import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow


In [ ]:
!ls -lh


In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv


In [ ]:

import pandas as pd

data = pd.read_csv("fer2013.csv")
print(data.head())


In [ ]:
# # ใช้เฉพาะ 'emotion' กับ 'pixels'
# emotions = data['emotion'].values
# pixels = data['pixels'].values

# X = []
# for pixel_sequence in pixels:
#     img = np.array([int(p) for p in pixel_sequence.split()]).reshape(48,48)
#     img = cv2.resize(img, (96,96))  # MobileNetV2 input size
#     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to RGB
#     X.append(img)
# X = np.array(X, dtype='float32') / 255.0
# y = to_categorical(emotions, num_classes=7)

# # แบ่ง train/test
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X_train.shape, y_train.shape)
# X = []
# y = []

# for idx, pixel_sequence in enumerate(pixels):
#     if type(pixel_sequence) != str or len(pixel_sequence.strip()) == 0:
#         continue  # skip empty or invalid pixels

#     arr = np.array([int(p) for p in pixel_sequence.split()])

#     if arr.size != 48*48:
#         continue  # skip row ที่มีจำนวน pixel ไม่ครบ

#     img = arr.reshape(48,48)
#     img = cv2.resize(img, (96,96))  # MobileNetV2 input size
#     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     X.append(img)
#     y.append(emotions[idx])

# X = np.array(X, dtype='float32') / 255.0
# y = to_categorical(y, num_classes=7)


In [ ]:
# ใช้เฉพาะ 'emotion' กับ 'pixels'
emotions = data['emotion'].values
pixels = data['pixels'].values

In [ ]:
# X = []
# y_clean = []

# for idx, pixel_sequence in enumerate(pixels):
#     # ตรวจสอบว่ามีค่า pixels จริง ๆ
#     if type(pixel_sequence) != str or len(pixel_sequence.strip()) == 0:
#         continue

#     # แปลงเป็น int array
#     arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])

#     # ตรวจสอบว่ามีจำนวน pixel ครบ 48*48
#     if arr.size != 48*48:
#         continue

#     # reshape & resize
#     img = arr.reshape(48,48)
#     img = cv2.resize(img, (96,96))  # MobileNetV2 input size
#     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

#     X.append(img)
#     y_clean.append(emotions[idx])

# X = np.array(X, dtype='float32') / 255.0
# y = to_categorical(y_clean, num_classes=7)

# print("Dataset ready:", X.shape, y.shape)
X = []
y_clean = []

for idx, pixel_sequence in enumerate(pixels):
    # skip if pixel_sequence missing or invalid
    if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
        continue

    arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])

    # check correct number of pixels
    if arr.size != 48 * 48:
        continue

    img_gray = arr.reshape(48, 48)

    # ensure dtype is float32 or uint8
    img_gray = img_gray.astype('float32')

    # now resize
    img_resized = cv2.resize(img_gray, (96, 96))

    # convert from gray to RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)

    X.append(img_rgb)
    y_clean.append(emotions[idx])

X = np.array(X, dtype='float32') / 255.0
y = tf.keras.utils.to_categorical(y_clean, num_classes=7)
print("Dataset ready:", X.shape, y.shape)


In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow as tf

base = tf.keras.applications.MobileNetV2(
    input_shape=(96,96,3),
    include_top=False,
    weights='imagenet'
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
emotion_model.summary()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

emotion_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,  # เพิ่มได้ตามกำลังเครื่อง
    batch_size=32
)


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (96,96))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
img = cv2.imread(file_name)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = detector.process(rgb)

if results.detections:
    for detection in results.detections:
        box = detection.location_data.relative_bounding_box
        h, w, _ = img.shape
        x1 = max(0, int(box.xmin * w))
        y1 = max(0, int(box.ymin * h))
        x2 = min(w, int((box.xmin + box.width) * w))
        y2 = min(h, int((box.ymin + box.height) * h))

        face_crop = img[y1:y2, x1:x2]
        emotion, score = predict_emotion(face_crop)

        cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)

cv2_imshow(img)


# New solation > Your session crashed after using all available RAM.

In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow


In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv

data = pd.read_csv("fer2013.csv")
print("Total rows:", len(data))
# data.head()


In [ ]:
subset_size = 8000
data_subset = data.sample(n=subset_size, random_state=42).reset_index(drop=True)

emotions = data_subset['emotion'].values
pixels = data_subset['pixels'].values

X = []
y_clean = []

for idx, pixel_sequence in enumerate(pixels):
    if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
        continue
    arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
    if arr.size != 48*48:
        continue
    img_gray = arr.reshape(48,48).astype('float32')
    img_resized = cv2.resize(img_gray, (48,48))  # smaller size to save RAM
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    X.append(img_rgb)
    y_clean.append(emotions[idx])

X = np.array(X, dtype='float32') / 255.0
y = to_categorical(y_clean, num_classes=7)
print("Dataset ready:", X.shape, y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train:", X_train.shape, "Validation:", X_val.shape)


In [ ]:
base = tf.keras.applications.MobileNetV2(
    input_shape=(48,48,3),
    include_top=False,
    weights='imagenet'
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
emotion_model.summary()


In [ ]:
emotion_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=32
)


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (48,48))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
def main():
  from google.colab import files
  uploaded = files.upload()

  file_name = list(uploaded.keys())[0]
  img = cv2.imread(file_name)
  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = detector.process(rgb)

  if results.detections:
      for detection in results.detections:
          box = detection.location_data.relative_bounding_box
          h, w, _ = img.shape
          x1 = max(0, int(box.xmin * w))
          y1 = max(0, int(box.ymin * h))
          x2 = min(w, int((box.xmin + box.width) * w))
          y2 = min(h, int((box.ymin + box.height) * h))

          face_crop = img[y1:y2, x1:x2]
          emotion, score = predict_emotion(face_crop)

          cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
          cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
          print(emotion, score)
  cv2_imshow(img)


In [ ]:
main()

In [ ]:
main()

In [ ]:
main()

In [ ]:
main()

In [ ]:
main()

In [ ]:
main()

# add data

In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow


In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv

data = pd.read_csv("fer2013.csv")
print("Total rows:", len(data))
# data.head()


In [ ]:
subset_size = int(len(data)/2)
data_subset = data.sample(n=subset_size, random_state=42).reset_index(drop=True)

emotions = data_subset['emotion'].values
pixels = data_subset['pixels'].values

X = []
y_clean = []

for idx, pixel_sequence in enumerate(pixels):
    if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
        continue
    arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
    if arr.size != 48*48:
        continue
    img_gray = arr.reshape(48,48).astype('float32')
    img_resized = cv2.resize(img_gray, (48,48))  # smaller size to save RAM
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    X.append(img_rgb)
    y_clean.append(emotions[idx])

X = np.array(X, dtype='float32') / 255.0
y = to_categorical(y_clean, num_classes=7)
print("Dataset ready:", X.shape, y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train:", X_train.shape, "Validation:", X_val.shape)

base = tf.keras.applications.MobileNetV2(
    input_shape=(48,48,3),
    include_top=False,
    weights='imagenet'
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
emotion_model.summary()


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (48,48))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
emotion_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32
)


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (48,48))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
def main():
  from google.colab import files
  uploaded = files.upload()

  file_name = list(uploaded.keys())[0]
  img = cv2.imread(file_name)
  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = detector.process(rgb)

  if results.detections:
      for detection in results.detections:
          box = detection.location_data.relative_bounding_box
          h, w, _ = img.shape
          x1 = max(0, int(box.xmin * w))
          y1 = max(0, int(box.ymin * h))
          x2 = min(w, int((box.xmin + box.width) * w))
          y2 = min(h, int((box.ymin + box.height) * h))

          face_crop = img[y1:y2, x1:x2]
          emotion, score = predict_emotion(face_crop)

          cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
          cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
          print(emotion, score)
  cv2_imshow(img)


In [ ]:
# ตั้งชื่อgit (เพื่อให้ commit ขึ้นชื่อคุณ)
!git config --global user.email "aralemook@gmail.com"
!git config --global user.name "Thanwarin"


In [ ]:
!git clone https://github.com/Thanwarin/robot-webots.git

In [ ]:
%cd /content/robot-webots

In [ ]:
!mkdir -p models

In [ ]:
%cd /content/robot-webots/models

In [ ]:
!mkdir -p emotion_detection

In [ ]:
emotion_model.save("/content/emotion_detection_pretrain_RetinaFace_FERplus.h5")

In [ ]:
%cd /content/robot-webots

In [ ]:
!mkdir -p /content/robot-webots/tmp/models/emotion_detection

In [ ]:
!cp /content/emotion_detection_pretrain_RetinaFace_FERplus.h5 models/emotion_detection/

In [ ]:
!ls -lh models

In [ ]:
# # 1. ตรวจสอบว่าคุณอยู่บน Branch tmp หรือไม่
# !git checkout tmp

# # 2. บอก Git ให้ติดตามไฟล์ใหม่ที่เพิ่งถูกคัดลอกเข้ามา
# !git add .

# # 3. บันทึกการเปลี่ยนแปลง (Commit)
# # *สำคัญ:* ข้อความ Commit ควรบอกให้ชัดเจนว่ามีการเพิ่มไฟล์ Model
# !git commit -m "feat(tmp): Add pre-trained emotion_detection_pretrain_RetinaFace_FERplus  model file"

# # 4. ส่ง (Push) การเปลี่ยนแปลงไปยัง GitHub
# !git push origin tmp

In [ ]:
# # 1. ย้ายไปยังโฟลเดอร์หลักของ Repository
# %cd /content/robot-webots

In [ ]:
# # 2. สร้าง Branch ใหม่ชื่อ tmp และย้ายไปที่ Branch นั้นทันที
# # ใช้ -b เพื่อสร้าง Branch ถ้ายังไม่มี
# # ถ้าคุณมีโฟลเดอร์/ไฟล์ชื่อ tmp อยู่แล้ว การใช้คำสั่งนี้อาจยังทำให้เกิด warning แต่ยังทำงานได้
# !git checkout -b tmp

In [ ]:

# # 3. จัดการไฟล์และ Add
# # ตรวจสอบว่าไฟล์ model อยู่ในตำแหน่ง tmp/models/emotion_detection/ แล้ว
# !git add .

In [ ]:
# # 3. บันทึกการเปลี่ยนแปลง (Commit)
# # *สำคัญ:* ข้อความ Commit ควรบอกให้ชัดเจนว่ามีการเพิ่มไฟล์ Model
# !git commit -m "feat(tmp): Add pre-trained emotion_detection_pretrain_RetinaFace_FERplus  model file"

In [ ]:
# # 5. Push และตั้งค่าการติดตาม (เพื่อส่ง Branch tmp ขึ้น GitHub)
# # คำสั่งนี้จะสร้าง Branch tmp บน GitHub และเชื่อมโยงกับการทำงานในเครื่อง
# !git push -u origin tmp

In [ ]:
# # 1. ย้ายไปยังโฟลเดอร์หลักของ Repository
# %cd /content/robot-webots

# # 2. สร้าง Branch ใหม่ชื่อ tmp และย้ายไปที่ Branch นั้นทันที
# # ใช้ -b เพื่อสร้าง Branch ถ้ายังไม่มี
# # ถ้าคุณมีโฟลเดอร์/ไฟล์ชื่อ tmp อยู่แล้ว การใช้คำสั่งนี้อาจยังทำให้เกิด warning แต่ยังทำงานได้
# !git checkout -b tmp

# # 3. จัดการไฟล์และ Add
# # ตรวจสอบว่าไฟล์ model อยู่ในตำแหน่ง tmp/models/emotion_detection/ แล้ว
# !git add .

# # 4. Commit การเปลี่ยนแปลง
# !git commit -m "feat(tmp): Add model file"

# # 5. Push และตั้งค่าการติดตาม (เพื่อส่ง Branch tmp ขึ้น GitHub)
# # คำสั่งนี้จะสร้าง Branch tmp บน GitHub และเชื่อมโยงกับการทำงานในเครื่อง
# !git push -u origin tmp

In [ ]:
# 1. ย้ายไปยังโฟลเดอร์หลักของ Repository
# **สำคัญมาก** คำสั่ง Git ทั้งหมดต้องรันจากที่นี่
%cd /content/robot-webots

# 2. ตรวจสอบว่า Branch 'tmp' ถูกสร้างและ Check Out ถูกต้อง
# ใช้ -- เพื่อ disambiguate (แก้ปัญหาชื่อซ้ำซ้อน)
# ถ้า Branch 'tmp' ยังไม่มีบน Local จะสร้างให้
!git checkout -b tmp

# 3. สร้างโครงสร้างโฟลเดอร์ปลายทาง
# โฟลเดอร์จะถูกสร้างภายใน /content/robot-webots/
!mkdir -p tmp/models/emotion_detection

# 4. คัดลอกไฟล์ Model เข้าไปยังโครงสร้างที่สร้างใหม่
# คัดลอกจาก /content/ ไปยัง /content/robot-webots/tmp/...
!cp /content/emotion_detection_pretrain_RetinaFace_FERplus.h5 tmp/models/emotion_detection/

# 5. บอก Git ให้ติดตามไฟล์ใหม่
!git add .

# 6. Commit การเปลี่ยนแปลง
!git commit -m "feat(tmp): Add pre-trained emotion_detection_pretrain_RetinaFace_FERplus  model file"

# 7. Push Branch 'tmp' ขึ้นไปยัง GitHub (รวมถึงโครงสร้างโฟลเดอร์ใหม่)
# -u คือการตั้งค่า upstream tracking เป็นการ Push ครั้งแรกของ Branch นี้
!git push -u origin tmp

# add data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow


In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv

data = pd.read_csv("fer2013.csv")
print("Total rows:", len(data))
# data.head()


In [ ]:
# subset_size = int(len(data)/2)
subset_size = len(data)
data_subset = data.sample(n=subset_size, random_state=42).reset_index(drop=True)

emotions = data_subset['emotion'].values
pixels = data_subset['pixels'].values

X = []
y_clean = []

for idx, pixel_sequence in enumerate(pixels):
    if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
        continue
    arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
    if arr.size != 48*48:
        continue
    img_gray = arr.reshape(48,48).astype('float32')
    img_resized = cv2.resize(img_gray, (48,48))  # smaller size to save RAM
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
    X.append(img_rgb)
    y_clean.append(emotions[idx])

X = np.array(X, dtype='float32') / 255.0
y = to_categorical(y_clean, num_classes=7)
print("Dataset ready:", X.shape, y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train:", X_train.shape, "Validation:", X_val.shape)

base = tf.keras.applications.MobileNetV2(
    input_shape=(48,48,3),
    include_top=False,
    weights='imagenet'
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
emotion_model.summary()


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (48,48))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
emotion_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=32
)


In [ ]:
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def predict_emotion(face_img):
    face_img = cv2.resize(face_img, (48,48))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = emotion_model.predict(face_img, verbose=0)
    emotion = emotion_labels[np.argmax(preds)]
    return emotion, np.max(preds)


In [ ]:
def main():
  from google.colab import files
  uploaded = files.upload()

  file_name = list(uploaded.keys())[0]
  img = cv2.imread(file_name)
  rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = detector.process(rgb)

  if results.detections:
      for detection in results.detections:
          box = detection.location_data.relative_bounding_box
          h, w, _ = img.shape
          x1 = max(0, int(box.xmin * w))
          y1 = max(0, int(box.ymin * h))
          x2 = min(w, int((box.xmin + box.width) * w))
          y2 = min(h, int((box.ymin + box.height) * h))

          face_crop = img[y1:y2, x1:x2]
          emotion, score = predict_emotion(face_crop)

          cv2.putText(img, f"{emotion} {score:.2f}", (x1, y1-10),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
          cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
          print(emotion, score)
  cv2_imshow(img)


In [ ]:
emotion_model.save('/content/drive/MyDrive/emotion_detection_model.h5')

In [ ]:
emotion_model.save("/content/emotion_detection_pretrain_RetinaFace_FERplus_ver2.h5")

In [ ]:
import os

save_path = "/content/drive/MyDrive/ml_models"
os.makedirs(save_path, exist_ok=True)

emotion_model.save(f"{save_path}/emotion_model.h5")


# truning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
print(tf.__version__,  np.__version__)

In [ ]:
import mediapipe as mp

In [ ]:
print(mp.__version__)

## Need a lot of RAM

In [ ]:

# # -----------------------------
# # Load FER2013 CSV
# # -----------------------------
# data = pd.read_csv('fer2013.csv')

# # -----------------------------
# # Preprocess
# # -----------------------------
# subset_size = len(data)
# data_subset = data.sample(n=subset_size, random_state=42).reset_index(drop=True)

# emotions = data_subset['emotion'].values
# pixels = data_subset['pixels'].values

# X = []
# y_clean = []

# IMG_SIZE = 96

# for idx, pixel_sequence in enumerate(pixels):
#     if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
#         continue
#     arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
#     if arr.size != 48*48:
#         continue
#     img_gray = arr.reshape(48,48).astype('float32')
#     img_resized = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))
#     img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
#     X.append(img_rgb)
#     y_clean.append(emotions[idx])

# X = np.array(X, dtype='float32') / 255.0
# y = to_categorical(y_clean, num_classes=7)

# # -----------------------------
# # Train/Validation split
# # -----------------------------
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y_clean
# )

# print("Train:", X_train.shape, "Validation:", X_val.shape)

# # -----------------------------
# # Data Augmentation
# # -----------------------------
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True
# )

# # -----------------------------
# # Build Model (MobileNetV2 pretrained)
# # -----------------------------
# base = tf.keras.applications.MobileNetV2(
#     input_shape=(IMG_SIZE,IMG_SIZE,3),
#     include_top=False,
#     weights='imagenet'
# )

# # Fine-tune top layers
# for layer in base.layers[:-20]:
#     layer.trainable = False
# for layer in base.layers[-20:]:
#     layer.trainable = True

# x = GlobalAveragePooling2D()(base.output)
# x = Dense(128, activation='relu')(x)
# output = Dense(7, activation='softmax')(x)

# emotion_model = Model(base.input, output)

# # -----------------------------
# # Compile Model
# # -----------------------------
# emotion_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

# emotion_model.summary()

# # -----------------------------
# # Callbacks
# # -----------------------------
# es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# # -----------------------------
# # Train Model
# # -----------------------------
# emotion_model.fit(
#     datagen.flow(X_train, y_train, batch_size=16),
#     validation_data=(X_val, y_val),
#     epochs=100,
#     callbacks=[es, rlr]
# )

# # -----------------------------
# # Emotion labels & prediction function
# # -----------------------------
# emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# def predict_emotion(face_img):
#     face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
#     face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
#     face_img = face_img.astype('float32') / 255.0
#     face_img = np.expand_dims(face_img, axis=0)
#     preds = emotion_model.predict(face_img, verbose=0)
#     emotion = emotion_labels[np.argmax(preds)]
#     return emotion, np.max(preds)


## Sol: a bit

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
import cv2

# -----------------------------
# Load FER2013 CSV
# -----------------------------
data = pd.read_csv('fer2013.csv')

# -----------------------------
# Preprocessing generator function
# -----------------------------
IMG_SIZE = 96
NUM_CLASSES = 7

def preprocess_generator(df, batch_size=16):
    n = len(df)
    while True:
        df = df.sample(frac=1).reset_index(drop=True)  # shuffle
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_df = df[start:end]
            X_batch = []
            y_batch = []
            for idx, row in batch_df.iterrows():
                pixel_sequence = row['pixels']
                emotion = row['emotion']
                if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
                    continue
                arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
                if arr.size != 48*48:
                    continue
                img_gray = arr.reshape(48,48).astype('float32')
                img_resized = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))
                img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                X_batch.append(img_rgb / 255.0)
                y_batch.append(emotion)
            if X_batch:
                X_batch = np.array(X_batch, dtype='float32')
                y_batch = to_categorical(y_batch, num_classes=NUM_CLASSES)
                yield X_batch, y_batch

# Split dataset
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['emotion'])

# -----------------------------
# Data Augmentation
# -----------------------------
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow(
    x=np.zeros((1,IMG_SIZE,IMG_SIZE,3)),  # dummy, will be replaced in generator
    y=np.zeros((1,NUM_CLASSES)),
    batch_size=16
)
# We'll use our generator function with flow
train_gen = preprocess_generator(train_df, batch_size=16)
val_gen = preprocess_generator(val_df, batch_size=16)

# -----------------------------
# Build Model
# -----------------------------
base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE,IMG_SIZE,3),
    include_top=False,
    weights='imagenet'
)
for layer in base.layers[:-20]:
    layer.trainable = False
for layer in base.layers[-20:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# Callbacks
# -----------------------------
es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# -----------------------------
# Train Model
# -----------------------------
steps_per_epoch = len(train_df) // 16
validation_steps = len(val_df) // 16

emotion_model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[es, rlr]
)


In [ ]:
emotion_model.save('/content/drive/MyDrive/emotion_detection_model.h5')

In [ ]:
emotion_model.save("/content/emotion_detection_pretrain_RetinaFace_FERplus_ver3.h5")

In [ ]:
import os

save_path = "/content/drive/MyDrive/ml_models"
os.makedirs(save_path, exist_ok=True)

emotion_model.save(f"{save_path}/emotion_model_v3.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
import cv2
import os



# -----------------------------
# Load FER2013 CSV
# -----------------------------
data = pd.read_csv('fer2013.csv')

# -----------------------------
# Preprocessing generator function
# -----------------------------
IMG_SIZE = 96
NUM_CLASSES = 7
ver_model = 'ver4'

def preprocess_generator(df, batch_size=16):
    n = len(df)
    while True:
        df = df.sample(frac=1).reset_index(drop=True)  # shuffle
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch_df = df[start:end]
            X_batch = []
            y_batch = []
            for idx, row in batch_df.iterrows():
                pixel_sequence = row['pixels']
                emotion = row['emotion']
                if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
                    continue
                arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
                if arr.size != 48*48:
                    continue
                img_gray = arr.reshape(48,48).astype('float32')
                img_resized = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))
                img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                X_batch.append(img_rgb / 255.0)
                y_batch.append(emotion)
            if X_batch:
                X_batch = np.array(X_batch, dtype='float32')
                y_batch = to_categorical(y_batch, num_classes=NUM_CLASSES)
                yield X_batch, y_batch

# Split dataset
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['emotion'])

# -----------------------------
# Data Augmentation
# -----------------------------
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow(
    x=np.zeros((1,IMG_SIZE,IMG_SIZE,3)),  # dummy, will be replaced in generator
    y=np.zeros((1,NUM_CLASSES)),
    batch_size=16
)
# We'll use our generator function with flow
train_gen = preprocess_generator(train_df, batch_size=16)
val_gen = preprocess_generator(val_df, batch_size=16)

# -----------------------------
# Build Model
# -----------------------------
base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE,IMG_SIZE,3),
    include_top=False,
    weights='imagenet'
)
for layer in base.layers[:-20]:
    layer.trainable = False
for layer in base.layers[-20:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

emotion_model = Model(base.input, output)
emotion_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# Callbacks
# -----------------------------
es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# -----------------------------
# Train Model
# -----------------------------
steps_per_epoch = len(train_df) // 16
validation_steps = len(val_df) // 16

emotion_model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=20,
    callbacks=[es, rlr]
)
emotion_model.save(f'/content/drive/MyDrive/emotion_detection_model_{ver_model}.h5')
emotion_model.save(f"/content/emotion_detection_pretrain_RetinaFace_FERplus_{ver_model}.h5")


save_path = "/content/drive/MyDrive/ml_models"
os.makedirs(save_path, exist_ok=True)

emotion_model.save(f"{save_path}/emotion_model_{ver_model}.h5")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install tensorflow keras opencv-python mediapipe --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
!wget -q https://gitlab.ecs.vuw.ac.nz/harisushehu/emotion-recognition-using-cnn/-/raw/master/dataset/fer2013.csv -O fer2013.csv


In [ ]:
# # ==============================
# # Imports
# # ==============================
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from sklearn.utils.class_weight import compute_class_weight
# import pandas as pd
# import numpy as np
# import cv2
# import os

# # ==============================
# # Config
# # ==============================
# IMG_SIZE = 96          # Input size for MobileNetV2
# NUM_CLASSES = 7
# BATCH_SIZE = 16
# EPOCHS = 50
# MODEL_NAME = "emotion_model_ver5"
# SAVE_PATH = "/content/drive/MyDrive/ml_models"
# os.makedirs(SAVE_PATH, exist_ok=True)

# # ==============================
# # Load FER2013 CSV
# # ==============================
# data = pd.read_csv('fer2013.csv')
# train_df, val_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['emotion'])

# # ==============================
# # Compute class weights
# # ==============================
# classes = np.array(data['emotion'])
# class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
# class_weight_dict = dict(enumerate(class_weights))
# print("Class weights:", class_weight_dict)

# # ==============================
# # Preprocessing + Augmentation Generator
# # ==============================
# def preprocess_and_augment(df, img_size=IMG_SIZE, batch_size=BATCH_SIZE, augment=False):
#     n = len(df)
#     while True:
#         df_shuff = df.sample(frac=1).reset_index(drop=True)
#         for start in range(0, n, batch_size):
#             batch = df_shuff[start:start+batch_size]
#             X_batch, y_batch = [], []
#             for _, row in batch.iterrows():
#                 pixel_sequence = row['pixels']
#                 emotion = row['emotion']
#                 if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
#                     continue
#                 arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
#                 if arr.size != 48*48:
#                     continue
#                 img = arr.reshape(48,48).astype('float32')
#                 img = cv2.resize(img, (img_size,img_size))
#                 img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#                 img /= 255.0
#                 if augment:
#                     img = tf.image.random_flip_left_right(img)
#                     img = tf.image.random_brightness(img, 0.1)
#                     img = tf.image.random_contrast(img, 0.9, 1.1)
#                 X_batch.append(img)
#                 y_batch.append(emotion)
#             if X_batch:
#                 X_batch = np.array(X_batch, dtype='float32')
#                 y_batch = to_categorical(y_batch, num_classes=NUM_CLASSES)
#                 yield X_batch, y_batch

# train_gen = preprocess_and_augment(train_df, augment=True)
# val_gen = preprocess_and_augment(val_df, augment=False)

# steps_per_epoch = len(train_df)//BATCH_SIZE
# validation_steps = len(val_df)//BATCH_SIZE

# # ==============================
# # Build Model (MobileNetV2)
# # ==============================
# base = tf.keras.applications.MobileNetV2(
#     input_shape=(IMG_SIZE,IMG_SIZE,3),
#     include_top=False,
#     weights='imagenet'
# )

# # Fine-tune top layers
# for layer in base.layers[:-50]:
#     layer.trainable = False
# for layer in base.layers[-50:]:
#     layer.trainable = True

# x = GlobalAveragePooling2D()(base.output)
# x = Dense(128, activation='relu')(x)
# output = Dense(NUM_CLASSES, activation='softmax')(x)
# emotion_model = Model(base.input, output)

# emotion_model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-4),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )
# emotion_model.summary()

# # ==============================
# # Callbacks
# # ==============================
# es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
# checkpoint = ModelCheckpoint(
#     filepath=os.path.join(SAVE_PATH, MODEL_NAME + ".h5"),
#     monitor='val_accuracy',
#     save_best_only=True,
#     verbose=1
# )

# # ==============================
# # Train Model
# # ==============================
# history = emotion_model.fit(
#     train_gen,
#     steps_per_epoch=steps_per_epoch,
#     validation_data=val_gen,
#     validation_steps=validation_steps,
#     epochs=EPOCHS,
#     callbacks=[es, rlr, checkpoint],
#     class_weight=class_weight_dict
# )

# # ==============================
# # Save final model
# # ==============================
# emotion_model.save(os.path.join(SAVE_PATH, MODEL_NAME + "_final.h5"))
# print(f"Model saved at {SAVE_PATH}")


In [ ]:
# ==============================
# Imports
# ==============================
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import numpy as np
import cv2
import os

# ==============================
# Config
# ==============================
IMG_SIZE = 96
NUM_CLASSES = 7
BATCH_SIZE = 16
EPOCHS = 50
MODEL_NAME = "emotion_model_ver6"
SAVE_PATH = "/content/drive/MyDrive/ml_models"
os.makedirs(SAVE_PATH, exist_ok=True)

# ==============================
# Load FER2013 CSV
# ==============================
data = pd.read_csv('fer2013.csv')
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['emotion'])

# ==============================
# Compute class weights
# ==============================
classes = np.array(data['emotion'])
class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# ==============================
# Generator with sample weights
# ==============================
def preprocess_and_augment_with_sample_weights(df, img_size=IMG_SIZE, batch_size=BATCH_SIZE, augment=False):
    weights_per_class = class_weight_dict
    n = len(df)
    while True:
        df_shuff = df.sample(frac=1).reset_index(drop=True)
        for start in range(0, n, batch_size):
            batch = df_shuff[start:start+batch_size]
            X_batch, y_batch, sample_weight = [], [], []
            for _, row in batch.iterrows():
                pixel_sequence = row['pixels']
                emotion = row['emotion']
                if not isinstance(pixel_sequence, str) or len(pixel_sequence.strip()) == 0:
                    continue
                arr = np.array([int(p) for p in pixel_sequence.split() if p.strip().isdigit()])
                if arr.size != 48*48:
                    continue
                img = arr.reshape(48,48).astype('float32')
                img = cv2.resize(img, (img_size,img_size))
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                img /= 255.0
                if augment:
                    img = tf.image.random_flip_left_right(img)
                    img = tf.image.random_brightness(img, 0.1)
                    img = tf.image.random_contrast(img, 0.9,1.1)
                X_batch.append(img)
                y_batch.append(emotion)
                sample_weight.append(weights_per_class[emotion])
            if X_batch:
                X_batch = np.array(X_batch, dtype='float32')
                y_batch = to_categorical(y_batch, num_classes=NUM_CLASSES)
                yield X_batch, y_batch, np.array(sample_weight, dtype='float32')

train_gen = preprocess_and_augment_with_sample_weights(train_df, augment=True)
val_gen = preprocess_and_augment_with_sample_weights(val_df, augment=False)

steps_per_epoch = len(train_df)//BATCH_SIZE
validation_steps = len(val_df)//BATCH_SIZE

# ==============================
# Build Model (MobileNetV2)
# ==============================
base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE,IMG_SIZE,3),
    include_top=False,
    weights='imagenet'
)

# Fine-tune top layers
for layer in base.layers[:-50]:
    layer.trainable = False
for layer in base.layers[-50:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)
emotion_model = Model(base.input, output)

emotion_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
emotion_model.summary()

# ==============================
# Callbacks
# ==============================
es = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint(
    filepath=os.path.join(SAVE_PATH, MODEL_NAME + ".h5"),
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# ==============================
# Train Model
# ==============================
history = emotion_model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=[es, rlr, checkpoint]
)

# ==============================
# Save final model
# ==============================
emotion_model.save(os.path.join(SAVE_PATH, MODEL_NAME + "_final.h5"))
print(f"Model saved at {SAVE_PATH}")


Class weights: {0: np.float64(1.035072539009547), 1: np.float64(9.372420997649517), 2: np.float64(1.001115853488437), 3: np.float64(0.5703319930708962), 4: np.float64(0.8436258492207152), 5: np.float64(1.2810380524023703), 6: np.float64(0.8271562255105334)}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 96, 96, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 48, 48,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 48, 48,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 48, 48,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 48, 48,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 48, 48,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 48, 48,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 48, 48,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 48, 48,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 48, 48,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 48, 48,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 48, 48,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 49, 49,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 24, 24,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 24, 24,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 24, 24,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 24, 24,    │      2,304 │ block_1_depthwis

 Total params: 2,422,855 (9.24 MB)

 Trainable params: 2,019,975 (7.71 MB)

...

1794/1794 ━━━━━━━━━━━━━━━━━━━━ 622s 346ms/step - accuracy: 0.8011 - loss: 0.4859 - val_accuracy: 0.6218 - val_loss: 1.4905 - learning_rate: 5.0000e-05
Epoch 10/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.8440 - loss: 0.3875
Epoch 10: val_accuracy did not improve from 0.62175
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 620s 346ms/step - accuracy: 0.8440 - loss: 0.3875 - val_accuracy: 0.6110 - val_loss: 1.5117 - learning_rate: 5.0000e-05
Epoch 11/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.8651 - loss: 0.3367
Epoch 11: val_accuracy did not improve from 0.62175
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 624s 348ms/step - accuracy: 0.8651 - loss: 0.3367 - val_accuracy: 0.6067 - val_loss: 1.6856 - learning_rate: 5.0000e-05
Epoch 12/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.8837 - loss: 0.2903
Epoch 12: val_accuracy improved from 0.62175 to 0.62441, saving model to /content/drive/MyDrive/ml_models/emotion_model_ver6.h5


1794/1794 ━━━━━━━━━━━━━━━━━━━━ 624s 348ms/step - accuracy: 0.8837 - loss: 0.2903 - val_accuracy: 0.6244 - val_loss: 1.6933 - learning_rate: 5.0000e-05
Epoch 13/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.8919 - loss: 0.2690
Epoch 13: val_accuracy improved from 0.62441 to 0.63055, saving model to /content/drive/MyDrive/ml_models/emotion_model_ver6.h5


1794/1794 ━━━━━━━━━━━━━━━━━━━━ 620s 346ms/step - accuracy: 0.8919 - loss: 0.2690 - val_accuracy: 0.6306 - val_loss: 1.8087 - learning_rate: 5.0000e-05
Epoch 14/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.9224 - loss: 0.2062
Epoch 14: val_accuracy did not improve from 0.63055
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 597s 333ms/step - accuracy: 0.9224 - loss: 0.2062 - val_accuracy: 0.6287 - val_loss: 1.8079 - learning_rate: 2.5000e-05
Epoch 15/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.9309 - loss: 0.1760
Epoch 15: val_accuracy did not improve from 0.63055
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 624s 348ms/step - accuracy: 0.9309 - loss: 0.1760 - val_accuracy: 0.6187 - val_loss: 1.9610 - learning_rate: 2.5000e-05
Epoch 16/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.9424 - loss: 0.1525
Epoch 16: val_accuracy did not improve from 0.63055
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 609s 339ms/step - accuracy: 0.9424 - loss: 0.1526 - val_accuracy: 0.6206 - val_loss: 1.

1794/1794 ━━━━━━━━━━━━━━━━━━━━ 604s 337ms/step - accuracy: 0.9480 - loss: 0.1376 - val_accuracy: 0.6345 - val_loss: 2.1061 - learning_rate: 2.5000e-05
Epoch 18/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.9491 - loss: 0.1322
Epoch 18: val_accuracy did not improve from 0.63446
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 619s 345ms/step - accuracy: 0.9491 - loss: 0.1322 - val_accuracy: 0.6141 - val_loss: 2.0541 - learning_rate: 2.5000e-05
Epoch 19/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.9539 - loss: 0.1200
Epoch 19: val_accuracy did not improve from 0.63446
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 602s 336ms/step - accuracy: 0.9539 - loss: 0.1200 - val_accuracy: 0.6314 - val_loss: 2.0997 - learning_rate: 1.2500e-05
Epoch 20/50
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.9572 - loss: 0.1129
Epoch 20: val_accuracy did not improve from 0.63446
1794/1794 ━━━━━━━━━━━━━━━━━━━━ 600s 334ms/step - accuracy: 0.9572 - loss: 0.1129 - val_accuracy: 0.6296 - val_loss: 2.

Model saved at /content/drive/MyDrive/ml_models
